In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Conv1D, BatchNormalization, LeakyReLU, concatenate, GlobalAveragePooling1D, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, optimizers


# class TransformerBlock(layers.Layer):
#     def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
#         super(TransformerBlock, self).__init__()
#         self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
#         self.ffn = tf.keras.Sequential(
#             [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
#         )
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout1 = layers.Dropout(rate)
#         self.dropout2 = layers.Dropout(rate)

#     def call(self, inputs, training):
#         attn_output = self.att(inputs, inputs)
#         attn_output = self.dropout1(attn_output, training=training)
#         out1 = self.layernorm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=training)
#         return self.layernorm2(out1 + ffn_output)



# CNN Block
def cnn_block(x, filters, kernel_size, strides, activation):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    if activation == 'leaky_relu':
        x = LeakyReLU(alpha=0.1)(x)
    elif activation == 'relu':
        x = ReLU()(x)
    return x

# Parameters
num_frames = 45  # Adjust to match the fixed number of frames you've processed per video
num_features = 1024  # Feature dimension per frame
batch_size = 64
number_of_labels = 10

# Load the dataset
load_dir = './saved_dataset'
loaded_dataset = tf.data.experimental.load(load_dir)

# Shuffle and prepare the datasets
shuffled_dataset = loaded_dataset.shuffle(buffer_size=10000)
split_size = int(0.8 * len(shuffled_dataset))
train_dataset = shuffled_dataset.take(split_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
validation_dataset = shuffled_dataset.skip(split_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Input for the frame sequence
frame_input = Input(shape=(num_frames, num_features), name='frame_input')

# CNN Branch
cnn_branch = cnn_block(frame_input, filters=10, kernel_size=5, strides=1, activation='leaky_relu')
cnn_branch = cnn_block(cnn_branch, filters=10, kernel_size=5, strides=1, activation='leaky_relu')
cnn_branch = cnn_block(cnn_branch, filters=10, kernel_size=5, strides=1, activation='leaky_relu')
cnn_branch_output = GlobalAveragePooling1D()(cnn_branch)  # Aggregate the features from the CNN branch

# LSTM Branch
lstm_branch = LSTM(512, return_sequences=True)(frame_input)
lstm_branch = LSTM(256, return_sequences=True)(lstm_branch)
lstm_branch_output = LSTM(128, return_sequences=False)(lstm_branch)

# Transformer Branch
# transformer_branch = layers.Dense(512, activation="relu")(frame_input)  
# transformer_branch = TransformerBlock(512, 8, 64)(transformer_branch)
# transformer_branch = layers.GlobalAveragePooling1D()(transformer_branch)
# transformer_branch = layers.Dropout(0.5)(transformer_branch)
# transformer_branch_output = layers.Dense(128, activation="relu")(transformer_branch)

# Combine CNN and LSTM branch outputs and transformer branch output
combined_output = concatenate([cnn_branch_output, lstm_branch_output])

# Final Dense layers
x = Dense(128, activation='relu')(combined_output)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(number_of_labels, activation='sigmoid')(x)

# Create and compile the model
model = Model(inputs=frame_input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model with validation
model.fit(train_dataset, epochs=30, validation_data=validation_dataset)


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 frame_input (InputLayer)    [(None, 45, 1024)]           0         []                            
                                                                                                  
 conv1d_6 (Conv1D)           (None, 45, 10)               51210     ['frame_input[0][0]']         
                                                                                                  
 batch_normalization_6 (Bat  (None, 45, 10)               40        ['conv1d_6[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 leaky_re_lu_6 (LeakyReLU)   (None, 45, 10)               0         ['batch_normalization_6[

2024-03-10 17:21:07.975669: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


117/117 [==============================] - 15s 91ms/step - loss: 0.4006 - accuracy: 0.2100 - val_loss: 0.3817 - val_accuracy: 0.2873
Epoch 2/30
117/117 [==============================] - 11s 78ms/step - loss: 0.2842 - accuracy: 0.4612 - val_loss: 0.4826 - val_accuracy: 0.1214
Epoch 3/30
117/117 [==============================] - 10s 76ms/step - loss: 0.2149 - accuracy: 0.5933 - val_loss: 0.3516 - val_accuracy: 0.3545
Epoch 4/30
117/117 [==============================] - 11s 77ms/step - loss: 0.1920 - accuracy: 0.6392 - val_loss: 0.3039 - val_accuracy: 0.5333
Epoch 5/30
117/117 [==============================] - 11s 77ms/step - loss: 0.1757 - accuracy: 0.6747 - val_loss: 0.2128 - val_accuracy: 0.5252
Epoch 6/30
117/117 [==============================] - 11s 76ms/step - loss: 0.1613 - accuracy: 0.6997 - val_loss: 0.1435 - val_accuracy: 0.7084
Epoch 7/30
117/117 [==============================] - 10s 75ms/step - loss: 0.1499 - accuracy: 0.7199 - val_loss: 0.1156 - val_accuracy: 0.7567
Epo

In [7]:
# save model to .h5

model.save('model.h5')  # creates a HDF5 file 'my_model.h5'